In [6]:
%%capture
import warnings
warnings.filterwarnings("ignore")
#HRRR Directory must be placed in working directory with title format 'HRRR 2020-01-01' to be read in 
import nexradaws
import shutil
from datetime import datetime 
import contextlib
import matplotlib as mpl 
import matplotlib.pyplot as plt
import matplotlib.dates as md
import matplotlib.colors as mcolors
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.geodesic as cg
import pyart
from metpy.plots import USCOUNTIES
import os 
import numpy as np
import xarray as xr
import pandas as pd
import logging
from imp import reload
import traceback
from collections import Counter
import pickle
import glob
from statsmodels import robust
import matplotlib.colors as mcolors
import netCDF4
from netCDF4 import Dataset
import sys
import seaborn as sns

In [7]:
%%capture

tds_read_in = Dataset('/Users/Gavin/Projects/Radar Meteorology/Data/IMPACTS_2DSH-P3_20200207_sizedistributions_v01.nc')
tds_lat = tds_read_in['LAT'][:]
tds_lon = tds_read_in['LON'][:]
tds_alt = tds_read_in['GALT'][:]
tds_time = np.array(netCDF4.num2date(tds_read_in['time'][:], units = 'seconds since 2020-02-07 00:00:00'), dtype = 'datetime64[s]')
tds_nt = tds_read_in['NT'][:]/1000
tds_nt[tds_nt == 0] = np.nan
tds_nt[tds_nt < -998.0] = np.nan

p3_flight_times = [['2020-02-07 15:08:00', '2020-02-07 15:22:05', 1],
                   ['2020-02-07 15:30:10', '2020-02-07 15:48:35', 2],
                   ['2020-02-07 15:58:00', '2020-02-07 16:12:55', 3],
                   ['2020-02-07 16:21:15', '2020-02-07 16:43:05', 4],
                   ['2020-02-07 16:49:50', '2020-02-07 17:17:35', 5],
                   ['2020-02-07 17:26:00', '2020-02-07 17:52:15', 6],
                   ['2020-02-07 17:59:30', '2020-02-07 18:20:10', 7]]

flight_leg_altitudes = []
flight_leg_date_ranges = []
flight_leg_nt = []
flight_leg_times = []
flight_leg_lons_lats_start = []                   
flight_leg_lons_lats_end = []
flight_leg_lons = []
flight_leg_lats = []

for i in range(len(p3_flight_times)): 
    st = p3_flight_times[i][0]
    et = p3_flight_times[i][1]
    start_idx = np.where(tds_time == np.datetime64(st))[0][0]

    end_idx = np.where(tds_time==np.datetime64(et))[0][0]
    start_lon_lat = (tds_lon[start_idx],tds_lat[start_idx])
    end_lon_lat = (tds_lon[end_idx],tds_lat[end_idx])
    flight_leg_lons_lats_start.append(start_lon_lat)                  
    flight_leg_lons_lats_end.append(end_lon_lat)
    
    
    index_time_subset = np.where((tds_time > np.datetime64(st)) & (tds_time < np.datetime64(et)))[0]
    
    lons = tds_lon[index_time_subset]
    lats = tds_lat[index_time_subset]
    flight_leg_lons.append(lons)
    flight_leg_lats.append(lats)
    altitudes = tds_alt[index_time_subset]
    flight_leg_altitudes.append(altitudes)
    times = tds_time[index_time_subset]
    flight_leg_times.append(times)
    date_range = pd.date_range(start=st,end=et,periods=len(altitudes))
    flight_leg_date_ranges.append(date_range)
    nt = tds_nt[index_time_subset]
    flight_leg_nt.append(nt)
    
flight_leg_altitudes = np.array(flight_leg_altitudes)/1000

In [8]:
figpath = '/Users/Gavin/Projects/Radar Meteorology/Microphysics Figures/'

for i in range(len(p3_flight_times)):
    
    data = pd.DataFrame({'nt' : flight_leg_nt[i]},index = flight_leg_date_ranges[i])
    ax = data.groupby(lambda x: x.strftime("%H:%M")).boxplot(subplots=False, figsize=(20,10),return_type="axes")
    labels = [item.get_text()[1:6] for item in ax.get_xticklabels()]
    ax.set_xticklabels(labels)
    ax.set_title('NT Versus Time - Flight Leg '+str(i))
    plt.savefig(figpath + 'FlightLeg' + str(i+1) + '_box', dpi = 500)
    plt.close()


    data = pd.DataFrame({'nt':flight_leg_nt[i], 'time':flight_leg_date_ranges[i]})
    data.time = data.time.dt.strftime("%H:%M")
    plt.figure(figsize=(20,10))
    sns.lineplot(x="time", y="nt",err_style="band", ci='sd', estimator="median", data=data)
    plt.gca().set_title('NT Versus Time - Flight Leg '+str(i))
    plt.savefig(figpath + 'FlightLeg' + str(i+1) + '_line', dpi = 500)
    plt.close()

In [ ]:
#makes a map of all radar locations available to us througout the northeast

locs = pyart.io.nexrad_common.NEXRAD_LOCATIONS
fig = plt.figure(figsize=(15, 12))
proj = ccrs.Miller()
ax = fig.add_subplot(1, 1, 1, projection=proj)
extent = [-77, -71, 40, 45]
ax.set_extent(extent, crs=ccrs.PlateCarree())
ax.coastlines(resolution='10m', color='black', linewidth=0.5)
states_provinces = cfeature.NaturalEarthFeature(category='cultural', name='admin_1_states_provinces_lakes', scale='10m', facecolor='none')
roads = cfeature.NaturalEarthFeature(category='cultural', name='roads', scale='10m', facecolor='none')
ax.add_feature(states_provinces, edgecolor='black', linewidth=0.5)
ax.add_feature(USCOUNTIES.with_scale('5m'), edgecolor='black', linewidth=0.2)
ax.add_feature(roads, edgecolor='blue', linewidth=0.3)
plt.title('NEXRAD Radar Locations - Northeast', fontweight='bold')

for key in locs:
    
    lon = locs[key]['lon']
    lat = locs[key]['lat']
    name = key
    ax.scatter(lon,lat,marker='o', color='black', transform=ccrs.PlateCarree())
    lat_mod = lat + .2
    lon_mod = lon - .4

    if lon >= extent[0] and lon <= extent[1] and lat >= extent[2] and lat <= extent[3]:
        ax.text(lon_mod, lat_mod, name, color='black', fontsize=14, transform=ccrs.PlateCarree())
ax.plot(tds_lon,tds_lat,transform=ccrs.PlateCarree())



#for i in range(len(flight_leg_lons_lats_start)):
lons = flight_leg_lons[6]
lats = flight_leg_lats[6]
ax.plot(lons,lats,transform=ccrs.PlateCarree(),color='red')
plt.text(lons[len(lons)//2],lats[len(lats)//2]+.01,'Flight Leg 7',transform = ccrs.PlateCarree())


circle_points = cg.Geodesic().circle(lon=-74.06444, lat=42.58639, radius=225*1000, n_samples=1000, endpoint=False)
ax.plot(circle_points[:,0],circle_points[:,1],transform=ccrs.PlateCarree(),color='red')

circle_points = cg.Geodesic().circle(lon=-74.06444, lat=42.58639, radius=150*1000, n_samples=1000, endpoint=False)
ax.plot(circle_points[:,0],circle_points[:,1],transform=ccrs.PlateCarree(),color='magenta')

circle_points = cg.Geodesic().circle(lon=-74.06444, lat=42.58639, radius=50*1000, n_samples=1000, endpoint=False)
ax.plot(circle_points[:,0],circle_points[:,1],transform=ccrs.PlateCarree(),color='green')
plt.savefig('/Users/Gavin/Projects/Radar Meteorology/Flight Illustration Leg 7',dpi=500)